In [373]:
data = [i.strip('.\n').split(' contain ') for i in open('day7.txt').readlines()]
data = [[j.split(', ') for j in i] for i in data]

In [361]:
# Construct dictionary and do BFS for part 1
from collections import defaultdict
import re
pat = re.compile('(?:[\d+]) (.* bag)')
tree = defaultdict(list)
for i in data:
    # print(i)
    for j in i[1]:
        if j.startswith('no'):
            continue
        j = re.findall(pat, j)

        tree[j[0]].append(i[0][0][:-1])

def bfs(bags, init='shiny gold bag'):
    visited = []
    queue = [init]

    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.append(node)
            neighbors = bags[node]

            for neighbor in neighbors:
                queue.append(neighbor)
            
    return visited

len(bfs(tree))-1

In [370]:
# Construct inverse of part 1 and do BFS again, summing up total bags seen
from collections import defaultdict
import re
pat = re.compile('([\d+]) (.* bag)')
tree = defaultdict(list)
for i in data:
    # print(i)
    for j in i[1]:
        if j.startswith('no'):
            continue
        j = re.findall(pat, j)

        tree[i[0][0][:-1]].append(*j)

def numbags(bags, bag):
    loc_bags = bags.get(bag)  

    if not loc_bags:
        return 0
    else:
        n = 0
        for nbags, loc_bag in loc_bags:
            j = multadd(bags, loc_bag)
            n += j*int(nbags) + int(nbags)
    
    return n

numbags(tree, 'shiny gold bag')

In [ ]:
def _map_rules(self, rule, lhs, rhs, reactants, products, return_reaction_center):
        """Operator mapping"""
        rxn = Chem.rdChemReactions.ReactionFromSmarts(rule)
        reactants = reactants.split(';')
        cofactor_index_reactants = [i for i, r in enumerate(reactants) if r != 'Any']
        products = products.split(';')
        cofactor_index_products = [i for i, p in enumerate(products) if p != 'Any']
        # if number of reactants does not match reactant template
        if len(lhs) > reactants.count('Any'):
            repetitive_mols = set(lhs).intersection(set(rhs))
            while repetitive_mols:
                lhs.remove(sorted(repetitive_mols)[0])
                rhs.remove(sorted(repetitive_mols)[0])
                repetitive_mols = set(lhs).intersection(set(rhs))
        lhs_set = set()
        for lhs_perm in itertools.permutations(lhs):
            lhs_set.add(lhs_perm)
        for lhs_perm in lhs_set:
            lhs_temp = list(lhs_perm)
            for c in cofactor_index_reactants:
                if self.molfiles_path:
                    lhs_temp[c:c] = [Chem.MolToSmiles(Chem.MolFromMolFile(os.path.sep.join([self.molfiles_path, self.cofactor_name_dict[reactants[c]] + '.mol'])))]
                elif self.seed_dict:
                    lhs_temp[c:c] = [self.seed_dict[self.cofactor_name_dict[reactants[c]]]]
            # pruned MetaCyc
            try:
                lhs_tuple = tuple([Chem.MolFromSmiles(i) for i in lhs_temp])
                outputs = rxn.RunReactants(lhs_tuple)
            except:
                try:
                    lhs_tuple = tuple([Chem.MolFromSmiles(i, sanitize=False) for i in lhs_temp])
                    outputs = rxn.RunReactants(lhs_tuple)
                except:
                    continue
            # # pickaxe
            # lhs_tuple_list = []
            # for i in lhs_temp:
            #     try:
            #         temp_mol = Chem.MolFromSmiles(i)
            #         temp_mol = AllChem.AddHs(temp_mol)
            #         AllChem.Kekulize(temp_mol, clearAromaticFlags=True)
            #     except:
            #         temp_mol = Chem.MolFromSmiles(i, sanitize=False)
            #     lhs_tuple_list.append(temp_mol)
            # lhs_tuple = tuple(lhs_tuple_list)
            # outputs = rxn.RunReactants(lhs_tuple)
            for rxn_output in outputs:
                rhs_run = [Chem.MolToSmiles(rhs_mols) for rhs_mols in rxn_output]
                rhs_list = copy.deepcopy(rhs_run)
                for c in cofactor_index_products:
                    rhs_list.remove(rhs_run[c])
                # for all tautomer possibilities of clean rhs
                for rhs in postsanitize_smiles(rhs):
                    rhs = list(rhs)
                    for rhs_list in postsanitize_smiles(rhs_list):
                        # pruned MetaCyc
                        if sorted(list(rhs_list)) == sorted(rhs):
                            # lhs_index = [int(np.where(np.argsort(lhs) == i)[0]) for i in np.argsort(lhs_perm)]
                            # rhs_index = [int(np.where(np.argsort(rhs) == i)[0]) for i in np.argsort(rhs_list)]
                            lhs_index = [lhs.index(i) for i in lhs_perm]
                            rhs_index = [rhs.index(i) for i in rhs_list]
                            # return atom index of reaction center
                            if return_reaction_center:
                                # try to append lhs reactants
                                lhs_mols = []
                                for l in lhs_perm:
                                    lhs_mols.append(Chem.MolFromSmiles(l))
                                    if not lhs_mols[-1]:
                                        lhs_mols[-1] = Chem.MolFromSmiles(l, sanitize=False)
                                smarts_list, _ = get_smarts(rule)
                                smarts_list = [s for i, s in enumerate(smarts_list)
                                               if i not in cofactor_index_reactants]
                                # possible reaction center
                                temp_lhs_match = [Chem.MolFromSmiles(l, sanitize=False).GetSubstructMatches(
                                    Chem.MolFromSmarts(smarts_list[i])) for i, l in enumerate(lhs_perm)]
                                reaction_center_set = [set(itertools.chain(*l)) for l in temp_lhs_match]
                                lhs_all_matches = itertools.product(*temp_lhs_match)
                                # for all possible reaction centers
                                for lhs_match in lhs_all_matches:
                                    # iterate over all reactants
                                    for l_idx, match in enumerate(lhs_match):
                                        for protect in reaction_center_set[l_idx] - set(match):
                                            lhs_mols[l_idx].GetAtomWithIdx(protect).SetProp('_protected', '1')
                                    # add cofactors
                                    lhs_temp_mol = list(lhs_mols)
                                    for c in cofactor_index_reactants:
                                        if self.molfiles_path:
                                            lhs_temp_mol[c:c] = [Chem.MolFromMolFile(os.path.sep.join(
                                                [self.molfiles_path, self.cofactor_name_dict[reactants[c]] + '.mol']))]
                                        elif self.seed_dict:
                                            lhs_temp_mol[c:c] = [Chem.MolFromSmiles(
                                                self.seed_dict[self.cofactor_name_dict[reactants[c]]])]
                                    # for all possible reaction outcomes
                                    for rhs_rxn in rxn.RunReactants(tuple(lhs_temp_mol)):
                                        for rhs_smiles in postsanitize_smiles([Chem.MolToSmiles(r) for r in rhs_rxn]):
                                            # found match
                                            if tuple(r for i, r in enumerate(rhs_smiles)
                                                     if i not in cofactor_index_products) == rhs_list:
                                                return lhs_index, rhs_index, list(lhs_match)
                                    # else remove protection
                                    for l_idx, match in enumerate(lhs_match):
                                        for deprotect in reaction_center_set[l_idx] - set(match):
                                            lhs_mols[l_idx].GetAtomWithIdx(deprotect).ClearProp('_protected')
                            else:
                                return lhs_index, rhs_index